In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import torch
from PFNExperiments.LinearRegression.Models.Transformer import MLP
from PFNExperiments.LinearRegression.Models.ModelPosterior import ModelPosteriorFullGaussian2
from PFNExperiments.LinearRegression.Models.Transformer_CNF_DoubleCondition2 import TransformerDecoderConditionalDouble_parallel
from PFNExperiments.LinearRegression.Models.Transformer_CNF_LearnedBaseDistribution import TransformerReturnRepresentations, TransformerCNFConditionalDecoderDouble_parallel_learnedBaseDistribution
from PFNExperiments.Training.FlowMatching.CFMLossOTGaussianBase import CFMLossOTGaussianBase

In [23]:
P = 10
N = 100

In [24]:
# Initialize the pre-trained encoder model

encoder = TransformerReturnRepresentations(
    n_features= P+1,
    seq_len= N,
    d_model= 512,
    n_heads= 8,
    dim_feedforward=512*2,
    dropout_rate = 0.1,
    n_layers=6,
    n_skip_layers_final_heads = 2,
    n_output_units_per_head=[P, P*P, P]
)

encoder.load_state_dict(torch.load("C:/Users/arik_/Documents/Dokumente/Job_Clausthal/PFNs/Repository/PFNExperiments/LinearRegression/Models/TestModels/SavedBaseModel/model.pth", map_location=torch.device('cpu')))

c:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\.conda\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [25]:
# get the correct class to transform model predictions to posterior samples
model_posterior = ModelPosteriorFullGaussian2(cov_reg_factor = 1e-3, use_lowrank_normal = False, diag_transform = lambda x: x**2)

In [26]:
mlp_to_process_encoder_output = MLP(
    n_input_units = encoder.d_model,
    n_output_units= 1024,
    n_hidden_units= 1024,
    n_skip_layers= 3,
    dropout_rate = 0.1
)

mlp_to_process_time_conditioning = MLP(
    n_input_units = 1,
    n_output_units = 64,
    n_hidden_units= 64,
    n_skip_layers= 2,
    dropout_rate = 0.1
)

In [27]:
decoder = TransformerDecoderConditionalDouble_parallel(
    n_input_features= P,
    d_model_decoder= 256,
    d_model_encoder= encoder.d_model,
    n_heads= 8,
    d_ff = 256*2,
    dropout= 0.1,
    n_condition_features_a = mlp_to_process_encoder_output.n_output_units,
    n_condition_features_b = mlp_to_process_time_conditioning.n_output_units,
    n_layers= 1,
    use_positional_encoding=False,
    use_self_attention=False,
)

In [28]:
mlp_to_process_decoder_output = MLP(
    n_input_units = decoder.d_model_decoder,
    n_output_units= 512,
    n_hidden_units= P,
    n_skip_layers= 3,
    dropout_rate = 0.1
)

In [29]:
model = TransformerCNFConditionalDecoderDouble_parallel_learnedBaseDistribution(
    encoder = encoder,
    model_posterior = model_posterior,
    mlp_to_process_encoder_output = mlp_to_process_encoder_output,
    mlp_to_process_time_conditioning = mlp_to_process_time_conditioning,
    mlp_to_process_decoder_output = mlp_to_process_decoder_output,
    decoder = decoder,
    freeze_encoder=True,
)

In [30]:
loss_fun = CFMLossOTGaussianBase(
    sigma_min=1e-4
)

In [31]:
BATCH_SIZE = 4 
test_x = torch.randn(BATCH_SIZE, N, P+1)
test_z = torch.randn(BATCH_SIZE, P)
test_t = torch.randn(BATCH_SIZE, 1)

In [32]:
#z_t.shape, encoder_representation.shape, test_t.shape

In [35]:
encoder_prediction, encoder_representation = model.forward_encoder(test_x)

base_distribution_samples = model.get_base_distribution_samples(encoder_prediction)

z_t, z_0_b = loss_fun.psi_t_conditional_fun(
    z_0_a = base_distribution_samples,
    z_1 = test_z,
    t = test_t,
    z_0_b = None,
)


vector_field_prediction = model.forward_decoder(
    z = z_t,
    x_encoder= encoder_representation,
    condition_time= test_t
)

torch.Size([4, 1, 10])
torch.Size([4, 256])


AttributeError: 'tuple' object has no attribute 'shape'